In [12]:
using CSV, DataFrames, Statistics, Dates, Gadfly, GLM, Distributions, LinearAlgebra, ScikitLearn, Query
include("Utils.jl")

normalizeData (generic function with 1 method)

## Collecte et nettoyage de donnees

In [37]:
data_surverse = CSV.read("data/surverses.csv",missingstring="-99999")
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


In [38]:
data_surverse = filter(row -> month(row.DATE) > 4, data_surverse) 
data_surverse = filter(row -> month(row.DATE) < 11, data_surverse) 
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String⍰
1,0642-01D,2013-05-01,0,missing
2,0642-01D,2013-05-02,0,missing
3,0642-01D,2013-05-03,0,missing
4,0642-01D,2013-05-04,0,missing
5,0642-01D,2013-05-05,0,missing


In [39]:
raison = coalesce.(data_surverse[:,:RAISON],"Inconnue")
data_surverse[!,:RAISON] = raison
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE,RAISON
,String,Date,Int64⍰,String
1,0642-01D,2013-05-01,0,Inconnue
2,0642-01D,2013-05-02,0,Inconnue
3,0642-01D,2013-05-03,0,Inconnue
4,0642-01D,2013-05-04,0,Inconnue
5,0642-01D,2013-05-05,0,Inconnue


In [40]:
data_surverse = filter(row -> row.RAISON ∈ ["P","Inconnue","TS"], data_surverse) 
select!(data_surverse, [:NO_OUVRAGE, :DATE, :SURVERSE])
first(data_surverse,5)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64⍰
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


In [41]:
surverse_df = dropmissing(data_surverse, disallowmissing=true)
first(surverse_df,5)

,NO_OUVRAGE,DATE,SURVERSE
,String,Date,Int64
1,0642-01D,2013-05-01,0
2,0642-01D,2013-05-02,0
3,0642-01D,2013-05-03,0
4,0642-01D,2013-05-04,0
5,0642-01D,2013-05-05,0


In [42]:
data_precipitations = CSV.read("data/precipitations.csv",missingstring="-99999")
rename!(data_precipitations, Symbol("St-Hubert")=>:StHubert)
first(data_precipitations,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-01-01,0,0,0,0,0,missing
2,2013-01-01,1,0,0,0,0,missing
3,2013-01-01,2,0,0,0,0,missing
4,2013-01-01,3,0,0,0,0,missing
5,2013-01-01,4,0,0,0,0,missing


In [43]:
data_precipitations = filter(row -> month(row.date) > 4, data_precipitations) 
data_precipitations = filter(row -> month(row.date) < 11, data_precipitations)
rowMean(data_precipitations, 5, 2)
first(data_precipitations,5)

,date,heure,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,2013-05-01,0,0,0,0,0,0
2,2013-05-01,1,0,0,0,0,0
3,2013-05-01,2,0,0,0,0,0
4,2013-05-01,3,0,0,0,0,0
5,2013-05-01,4,0,0,0,0,0


## Analyse exploratoire

## Variable explicative 1 : Trop-Plein Z

In [44]:
data_ouvrage = CSV.read("data/ouvrages-surverses.csv",missingstring="")

# Extract speficic ouvrage
df_ouvrage = filter(row ->
    row.ID_ouvrage == "3260-01D" ||
    row.ID_ouvrage == "3350-07D" ||
    row.ID_ouvrage == "4240-01D" ||
    row.ID_ouvrage == "4350-01D" ||
    row.ID_ouvrage == "4380-01D", data_ouvrage)

df_ouvrage = df_ouvrage[:,[:ID_ouvrage, Symbol("Trop-Plein Z")]]
rename!(df_ouvrage, Symbol("ID_ouvrage")=>:NO_OUVRAGE)

meanHeight = mean(skipmissing(df_ouvrage[Symbol("Trop-Plein Z")]))

# Replace missing height with it's mean
df_ouvrage[ismissing.(df_ouvrage[Symbol("Trop-Plein Z")]), Symbol("Trop-Plein Z")] .= meanHeight

df_ouvrage

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[44]:13
└ @ Core In[44]:13
┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[44]:15
└ @ Core In[44]:15


,NO_OUVRAGE,Trop-Plein X,Trop-Plein Y,Trop-Plein Z
,String,Float64,Float64,Float64⍰
1,3260-01D,2.98543e5,5.05676e6,20.17
2,3350-07D,2.89801e5,5.04516e6,20.75
3,4240-01D,3.05755e5,5.05665e6,11.91
4,4350-01D,3.00503e5,5.03991e6,17.61
5,4380-01D,2.9982e5,5.03643e6,17.61


## Variable explicative 2 : Somme des précipitations par jour

In [45]:
# Faire la somme des precipitations par rapport a la date
pcp_sum = by(data_precipitations, :date,  McTavish = :McTavish=>sum, Bellevue = :Bellevue=>sum, 
           Assomption = :Assomption=>sum, Trudeau = :Trudeau=>sum, StHubert = :StHubert=>sum)

# maxMcTavish = findmax(convert(Array, pcp_sum[:,[2]]))[1]
# maxBellevue = findmax(convert(Array, pcp_sum[:,[3]]))[1]
# maxAssomption = findmax(convert(Array, pcp_sum[:,[4]]))[1]
# maxTrudeau = findmax(convert(Array, pcp_sum[:,[5]]))[1]
# maxStHubert = findmax(convert(Array, pcp_sum[:,[6]]))[1]

# pcp_sum[:McTavish] = [normalizeData(x, 0, maxMcTavish) for x in pcp_sum[:McTavish]]
# pcp_sum[:Bellevue] = [normalizeData(x, 0, maxMcTavish) for x in pcp_sum[:Bellevue]]
# pcp_sum[:Assomption] = [normalizeData(x, 0, maxMcTavish) for x in pcp_sum[:Assomption]]
# pcp_sum[:Trudeau] = [normalizeData(x, 0, maxMcTavish) for x in pcp_sum[:Trudeau]]
# pcp_sum[:StHubert] = [normalizeData(x, 0, maxMcTavish) for x in pcp_sum[:StHubert]]

first(pcp_sum,5)


,date,McTavish,Bellevue,Assomption,Trudeau,StHubert
,Date,Int64,Int64,Int64,Int64,Int64
1,2013-05-01,0,0,0,0,0
2,2013-05-02,0,0,0,0,0
3,2013-05-03,0,0,0,0,0
4,2013-05-04,0,0,0,0,0
5,2013-05-05,0,0,0,0,0


## Variable explicative 3 : Maximum horaire de précipitation par jour

In [46]:
# Trouver le max de precipitation journalier
pcp_max = by(data_precipitations, :date,  McTavish = :McTavish=>maximum, Bellevue = :Bellevue=>maximum, 
   Assomption = :Assomption=>maximum, Trudeau = :Trudeau=>maximum, StHubert = :StHubert=>maximum)

# maxMcTavish = findmax(convert(Array, pcp_max[:,[2]]))[1]
# maxBellevue = findmax(convert(Array, pcp_max[:,[3]]))[1]
# maxAssomption = findmax(convert(Array, pcp_max[:,[4]]))[1]
# maxTrudeau = findmax(convert(Array, pcp_max[:,[5]]))[1]
# maxStHubert = findmax(convert(Array, pcp_max[:,[6]]))[1]

# pcp_max[:McTavish] = [normalizeData(x, 0, maxMcTavish) for x in pcp_max[:McTavish]]
# pcp_max[:Bellevue] = [normalizeData(x, 0, maxMcTavish) for x in pcp_max[:Bellevue]]
# pcp_max[:Assomption] = [normalizeData(x, 0, maxMcTavish) for x in pcp_max[:Assomption]]
# pcp_max[:Trudeau] = [normalizeData(x, 0, maxMcTavish) for x in pcp_max[:Trudeau]]
# pcp_max[:StHubert] = [normalizeData(x, 0, maxMcTavish) for x in pcp_max[:StHubert]]

rename!(pcp_max,
    Symbol("McTavish")=>:McTavishMax,
    Symbol("Bellevue")=>:BellevueMax,
    Symbol("Assomption")=>:AssomptionMax,
    Symbol("Trudeau")=>:TrudeauMax,
    Symbol("StHubert")=>:StHubertMax)


,date,McTavishMax,BellevueMax,AssomptionMax,TrudeauMax,StHubertMax
,Date,Int64,Int64,Int64,Int64,Int64
1,2013-05-01,0,0,0,0,0
2,2013-05-02,0,0,0,0,0
3,2013-05-03,0,0,0,0,0
4,2013-05-04,0,0,0,0,0
5,2013-05-05,0,0,0,0,0
6,2013-05-06,0,0,0,0,0
7,2013-05-07,0,0,0,0,0
8,2013-05-08,0,0,0,0,0
9,2013-05-09,10,0,19,0,3


## Préparation pour créer le modèle

In [47]:
# Filtrer les ouvrages étudiés
df_surve = filter(row ->
    row.NO_OUVRAGE == "3260-01D" ||
    row.NO_OUVRAGE == "3350-07D" ||
    row.NO_OUVRAGE == "4240-01D" ||
    row.NO_OUVRAGE == "4350-01D" ||
    row.NO_OUVRAGE == "4380-01D", surverse_df)

rename!(df_surve, Symbol("DATE")=>:date)

# # Fusionner les datasets contenant les ouvrages etudies et les variables explicatives ensemble
enhanced_surve = join(df_surve, pcp_sum, on=:date, kind=:inner)
enhanced_surve = join(enhanced_surve, pcp_max, on=:date, kind=:inner)
enhanced_surve = join(enhanced_surve, df_ouvrage, on=:NO_OUVRAGE, kind=:left)

,NO_OUVRAGE,date,SURVERSE,McTavish,Bellevue,Assomption,Trudeau,StHubert
,String,Date,Int64,Int64,Int64,Int64,Int64,Int64
1,3260-01D,2013-05-01,0,0,0,0,0,0
2,3260-01D,2013-05-02,0,0,0,0,0,0
3,3260-01D,2013-05-03,0,0,0,0,0,0
4,3260-01D,2013-05-04,0,0,0,0,0,0
5,3260-01D,2013-05-05,0,0,0,0,0,0
6,3260-01D,2013-05-06,0,0,0,0,0,0
7,3260-01D,2013-05-07,0,0,0,0,0,0
8,3260-01D,2013-05-08,0,0,0,0,0,0
9,3260-01D,2013-05-09,0,10,0,19,0,5


In [48]:
# # Trouver les moyennes pour remplacer les donnees manquantes
# McTavish_Sum_Avg = mean(skipmissing(enhanced_surve[:,:McTavish]))
# Bellevue_Sum_Avg = mean(skipmissing(enhanced_surve[:,:Bellevue]))
# Assomption_Sum_Avg = mean(skipmissing(enhanced_surve[:,:Assomption]))
# Trudeau_Sum_Avg = mean(skipmissing(enhanced_surve[:,:Trudeau]))
# StHubert_Sum_Avg = mean(skipmissing(enhanced_surve[:,:StHubert]))

# McTavish_Max_Avg = mean(skipmissing(enhanced_surve[:,:McTavishMax]))
# Bellevue_Max_Avg = mean(skipmissing(enhanced_surve[:,:BellevueMax]))
# Assomption_Max_Avg = mean(skipmissing(enhanced_surve[:,:AssomptionMax]))
# Trudeau_Max_Avg = mean(skipmissing(enhanced_surve[:,:TrudeauMax]))
# StHubert_Max_Avg = mean(skipmissing(enhanced_surve[:,:StHubertMax]))

# # Tableau avec les moyennes pour remplacer les donnees manquantes
# replacement = [McTavish_Sum_Avg, Bellevue_Sum_Avg, Assomption_Sum_Avg, Trudeau_Sum_Avg, StHubert_Sum_Avg,
#                 McTavish_Max_Avg, Bellevue_Max_Avg, Assomption_Max_Avg, Trudeau_Max_Avg, StHubert_Max_Avg]

# # Remplacer les donnees manquantes
# # Commence a 4 pour les column des precipitations
# for i = 4:size(enhanced_surve,2)
#     for j = 1:size(enhanced_surve, 1)
#         if enhanced_surve[j,i] === missing
#             enhanced_surve[j,i] = floor(replacement[i - 3])
#         end
#     end
# end


enhanced_surve.NO_OUVRAGE[enhanced_surve.NO_OUVRAGE .== "3260-01D"] .= "1"
enhanced_surve.NO_OUVRAGE[enhanced_surve.NO_OUVRAGE .== "3350-07D"] .= "2"
enhanced_surve.NO_OUVRAGE[enhanced_surve.NO_OUVRAGE .== "4240-01D"] .= "3"
enhanced_surve.NO_OUVRAGE[enhanced_surve.NO_OUVRAGE .== "4350-01D"] .= "4"
enhanced_surve.NO_OUVRAGE[enhanced_surve.NO_OUVRAGE .== "4380-01D"] .= "5"

enhanced_surve[:NO_OUVRAGE] = [parse(Int,x) for x in enhanced_surve[:NO_OUVRAGE]] 

names(enhanced_surve)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[48]:34
└ @ Core In[48]:34
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[48]:34
└ @ Core In[48]:34


16-element Array{Symbol,1}:
 :NO_OUVRAGE           
 :date                 
 :SURVERSE             
 :McTavish             
 :Bellevue             
 :Assomption           
 :Trudeau              
 :StHubert             
 :McTavishMax          
 :BellevueMax          
 :AssomptionMax        
 :TrudeauMax           
 :StHubertMax          
 Symbol("Trop-Plein X")
 Symbol("Trop-Plein Y")
 Symbol("Trop-Plein Z")

## Générer le modèle

In [49]:
x = convert(Array, enhanced_surve[:,[1,4,5,6,7,8,9,10,11,12,13,14]])
y = convert(Array, enhanced_surve[:,:SURVERSE])

# Créer le modèle
@sk_import naive_bayes : MultinomialNB
nb_model = MultinomialNB()

# Entraîner le modèle
nb_model.fit(x,y)

# Evaluer l'accuracy du modèle
predictions = nb_model.predict(x)
@sk_import metrics : accuracy_score
accuracy = accuracy_score(predictions, y)
println("L'accuracy est de $accuracy")

using ScikitLearn.CrossValidation: cross_val_score
cross_val_score(MultinomialNB(), x,y, cv=5)

L'accuracy est de 0.8956911678689803

┌ Warning: `convert(::Type{Array}, df::AbstractDataFrame)` is deprecated, use `convert(Matrix, df)` instead.
│   caller = top-level scope at In[49]:1
└ @ Core In[49]:1


5-element Array{Float64,1}:
 0.8695228821811101
 0.8851022395326192
 0.9346341463414635
 0.895609756097561 
 0.8946341463414634

In [54]:
### Retrieve test data
test = CSV.read("data/test.csv");

# Filtrer les ouvrages étudiés
test_df_surve = filter(row ->
    row.NO_OUVRAGE == "3260-01D" ||
    row.NO_OUVRAGE == "3350-07D" ||
    row.NO_OUVRAGE == "4240-01D" ||
    row.NO_OUVRAGE == "4350-01D" ||
    row.NO_OUVRAGE == "4380-01D", test)

rename!(test_df_surve, Symbol("DATE")=>:date)

# # # Fusionner les datasets contenant les ouvrages etudies et les variables explicatives ensemble
test_enhanced_surve = join(test_df_surve, pcp_sum, on=:date, kind=:inner)
test_enhanced_surve = join(test_enhanced_surve, pcp_max, on=:date, kind=:inner)
test_enhanced_surve = join(test_enhanced_surve, df_ouvrage, on=:NO_OUVRAGE, kind=:left)

# McTavish_Sum_Avg = mean(skipmissing(test_enhanced_surve[:,:McTavish]))
# Bellevue_Sum_Avg = mean(skipmissing(test_enhanced_surve[:,:Bellevue]))
# Assomption_Sum_Avg = mean(skipmissing(test_enhanced_surve[:,:Assomption]))
# Trudeau_Sum_Avg = mean(skipmissing(test_enhanced_surve[:,:Trudeau]))
# StHubert_Sum_Avg = mean(skipmissing(test_enhanced_surve[:,:StHubert]))

# McTavish_Max_Avg = mean(skipmissing(test_enhanced_surve[:,:McTavishMax]))
# Bellevue_Max_Avg = mean(skipmissing(test_enhanced_surve[:,:BellevueMax]))
# Assomption_Max_Avg = mean(skipmissing(test_enhanced_surve[:,:AssomptionMax]))
# Trudeau_Max_Avg = mean(skipmissing(test_enhanced_surve[:,:TrudeauMax]))
# StHubert_Max_Avg = mean(skipmissing(test_enhanced_surve[:,:StHubertMax]))

# replacement = [McTavish_Sum_Avg, Bellevue_Sum_Avg, Assomption_Sum_Avg, Trudeau_Sum_Avg, StHubert_Sum_Avg,
#                 McTavish_Max_Avg, Bellevue_Max_Avg, Assomption_Max_Avg, Trudeau_Max_Avg, StHubert_Max_Avg]

# # Commence a 3 pour les column des precipitations
# for i = 3:size(test_enhanced_surve,2)
#     for j = 1:size(test_enhanced_surve, 1)
#         if test_enhanced_surve[j,i] === missing
#             test_enhanced_surve[j,i] = floor(replacement[i - 2])
#         end
#     end
# end

test_enhanced_surve.NO_OUVRAGE[test_enhanced_surve.NO_OUVRAGE .== "3260-01D"] .= "1"
test_enhanced_surve.NO_OUVRAGE[test_enhanced_surve.NO_OUVRAGE .== "3350-07D"] .= "2"
test_enhanced_surve.NO_OUVRAGE[test_enhanced_surve.NO_OUVRAGE .== "4240-01D"] .= "3"
test_enhanced_surve.NO_OUVRAGE[test_enhanced_surve.NO_OUVRAGE .== "4350-01D"] .= "4"
test_enhanced_surve.NO_OUVRAGE[test_enhanced_surve.NO_OUVRAGE .== "4380-01D"] .= "5"

test_enhanced_surve[:NO_OUVRAGE] = [parse(Int,x) for x in test_enhanced_surve[:NO_OUVRAGE]] 


test_enhanced_surve
x = convert(Array, test_enhanced_surve[:,[1,3,4,5,6,7,8,9,10,11,12,13]])

predictions = nb_model.predict(x)
n = size(test_df_surve, 1)

┌ Warning: `getindex(df::DataFrame, col_ind::ColumnIndex)` is deprecated, use `df[!, col_ind]` instead.
│   caller = top-level scope at In[54]:48
└ @ Core In[54]:48
┌ Warning: `setindex!(df::DataFrame, v::AbstractVector, col_ind::ColumnIndex)` is deprecated, use `begin
│     df[!, col_ind] = v
│     df
│ end` instead.
│   caller = top-level scope at In[54]:48
└ @ Core In[54]:48
┌ Warning: `convert(::Type{Array}, df::AbstractDataFrame)` is deprecated, use `convert(Matrix, df)` instead.
│   caller = top-level scope at In[54]:53
└ @ Core In[54]:53


283

In [56]:
ID = Array{String}(undef, n)
for i = 1:n
    ID[i] = string(test[i,:NO_OUVRAGE], "_", Dates.format(test[i,:DATE], "yyyy-mm-dd"))
end
YTotalBool = Array{String}(undef, n)

for i = 1:n
    if predictions[i] == 0
        YTotalBool[i] = "FALSE"
    else
        YTotalBool[i] = "TRUE"
    end
end

prediction = DataFrame(ID = ID, Surverse = YTotalBool)
CSV.write("naive-bayes-submission.csv",prediction)

"naive-bayes-submission.csv"